In [4]:
import time
start_time = time.time()

import os
import pandas as pd
import re
import numpy as np
import xml.etree.ElementTree as ET
from slugify import slugify
import requests
from openpyxl import load_workbook

# Determining the path where the files are located and creating a new folder
path = '/content/drive/MyDrive/DDC/13012022'
pathn = path + '/CO2_calculation'
try:
  os.mkdir(pathn)
except:
  pass
  
# Properties for which we want to collect data on the amount of volume
propstr = ['Area', 'Volume']

# Main function for grouping data and saving a file
def crtable(filename):
    filenamep = path + '/' + filename
    df = pd.read_csv(filenamep, low_memory=False)
    filedae = path + '/' + filename[:-8]+'dae'
    
    #  Fetching only numbers from string values of volumetric parameters
    propindf = []
    
    #grouping by element types for different formats
    if 'ifc' in filenamep:
      search_parameter = 'Type'
    else:
      search_parameter = 'Type Name'

    # Converting all "propstr" values in columns to numeric values
    for el in propstr:
      if el in df.columns:
        propindf.append(el)
    def find_number(text):
      num = re.findall(r'[0-9]+', text)
      return ".".join(num)
    for el in propindf:
      df[el] = df[el].astype(str)
      df[el] = df[el].apply(lambda x: find_number(x))
      df[el] = pd.to_numeric(df[el], errors='coerce')
      df[el] = df[el].replace(np.nan, 0)
      df[el] = df[el].replace('None', 0)
      df[el] = df[el].fillna(0)
    try:
        df[el] = df[el].astype(float)
    except:
        pass

    # Summation of all data that are grouped by search_parameter located in the propindf columns
    df1=pd.pivot_table(df, index=[search_parameter],values=propindf,aggfunc=np.sum)
    df1 = df1.add_prefix('Sum of ')

    # Determination of the number of elements in groups
    df2= df.groupby([search_parameter])[propindf[0]].agg(['count'])
    dfallpar = pd.DataFrame()  
    df['Unnamed: 0'] = df['Unnamed: 0'].astype(str)
    comma = lambda x: ', '.join(x.unique())
    df3 = df.groupby([search_parameter]) .agg({'Unnamed: 0': comma})
    
    # Collecting data into one dataframe
    dfallpar = pd.concat([df2, df1, df3], axis=1)
    dfallpar.rename(columns=({ 'Unnamed: 0': 'Id´s', 'count': 'Amount'}), inplace=True,)
    
    # Formula to add to the HLS table to automatically find values in CG for a group
    formeldba = "=IFERROR(INDEX(Info!$F$6:$F$17,MATCH($E"
    formeldbe = ",Info!$C$6:$C$17,0),MATCH(F$1,Info!$F$4:$F$4,0)),0)"

    # Creating unique formulas for each row
    forarr = []
    forarrm = []
    anzr = len(dfallpar.index) + 2
    for iteration, item in enumerate(range(2, anzr, 1)):
      forarr.append(formeldba + str(item) + formeldbe)
      forarrm.append("=D"+str(item)+"*F"+str(item))

    # Creating new columns in the Excel file
    dfallpar.insert(3, "The sum of the CO2 emissions for the group, Kg", forarrm)
    dfallpar.insert(3, "Сoefficient m3 to kg", forarr)
    dfallpar.insert(3, "CO2 group", '')
    
    # Use and download a sample excel file
    url = 'https://github.com/DataDrivenConstruction/CO2_calculating-the-embodied-carbon/raw/main/Sample_Fossil-Carbon-Emitted-in-Production.xlsx'
    r = requests.get(url)
    excelf = pathn + '/' + 'CO2_' + filename+'.xlsx'     
    with open(excelf, 'wb') as f:
        f.write(r.content)

    # Saving data to file
    book = load_workbook(excelf)
    writer = pd.ExcelWriter(excelf, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    dfallpar.to_excel(writer, 'CO2 group')
    writer.save()

    dfallpar['type'] = dfallpar.index
    dfallpar['type'] = dfallpar['type'].apply(slugify)
    print("File created: " + excelf)
    
    # Start sorting geometry from DAE file
    # Formation of a data tree from the DAE format
    daegrpath = pathn + '/' + 'DAEgroups_' + filename[:-9] 
    try:
      os.mkdir(daegrpath)
    except:
      pass

    # If the ID of an element from the group_ids_str list that was found earlier matches,
    # all elements with this ID are found in the DAE file, and all other elements are deleted
    filedaearr = []
    for index, row in dfallpar.iterrows():
      fileObject = open(filedae, "r")
      treeq = ET.parse(fileObject)
      root = treeq.getroot()
      ET.register_namespace("", "http://www.collada.org/2005/11/COLLADASchema")
      geom_list = []
      group_ids_str = []
      group_ids_str = re.findall(r'\d+', row['Id´s'])
      for node in root.findall('.//{http://www.collada.org/2005/11/COLLADASchema}node'):
          tree = treeq
          if node.attrib['id'] in group_ids_str:
            try:
              url = list(node)[0].get('url')
              geom_list.append(url[1:])
            except:
              pass
          else:
              try:
                  nd = node.find(
                      '{http://www.collada.org/2005/11/COLLADASchema}instance_geometry')
                  node.remove(nd)
              except:
                  0
      for geomet in root.findall('.//{http://www.collada.org/2005/11/COLLADASchema}geometry'):
            if geomet.attrib['id'] in geom_list:
                0
            else:
                try:
                  md = geomet.find(
                      '{http://www.collada.org/2005/11/COLLADASchema}mesh')
                  geomet.remove(md)
                except:
                  pass

      # Formation of a new name for the DAE file with grouped elements
      #words_pattern = '[a-zA-Z10-9]+'
      invalid = '<>:"/\|?* '
      for char in invalid:
        index = index.replace(char, '')
      regw = index + '.dae'
      filedaena = daegrpath + '/' + regw
      with open(filedaena, 'w') as f:
          tree.write(f, encoding='unicode')
      #filedaearr.append("""=HYPERLINK("["""+"/" + "DAEgroups_" + filename[:-9] + "/" + regw + "]" + regw +"""")""")
      filedaearr.append('=HYPERLINK(LEFT(CELL("filename",A1),FIND("[",CELL("filename",A1))-1)&"' + "DAEgroups_" + filename[:-9] + '\\' + regw +'","'+ regw + '")')
    dfallpar.drop(columns=['type'])
    dfallpar.insert(6, "Group geometry in DAE, file hyperlink *.dae)", filedaearr)
    with open(excelf, 'wb') as f:
        f.write(r.content)

    # Saving data to file
    book = load_workbook(excelf)
    writer = pd.ExcelWriter(excelf, engine='openpyxl') 
    writer.book = book
    writer.sheets = dict((ws.title, ws) for ws in book.worksheets)
    dfallpar.to_excel(writer, 'CO2 group')
    writer.save()

# Function execution cycle for all CSV files in the folder
for filename in os.listdir(path):
  if filename.endswith("csv"): 
      crtable(filename)
      
print("--- %s seconds ---" % (time.time() - start_time))
# Saving data to a ZIP file for downloading to a computer
#!zip -r /content/file.zip /content/IFC2/quick_rough_estimate

File created: /content/drive/MyDrive/BIMEXCEL/13012022/CO2_calculation/CO2_ID-White_R22_rvt.json.csv.xlsx
File created: /content/drive/MyDrive/BIMEXCEL/13012022/CO2_calculation/CO2_ID-White_R22_ifc.json.csv.xlsx
--- 34.02720499038696 seconds ---


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
